# 智谱 AI 流式输出测试

本 Notebook 演示智谱 AI 流式输出 (Streaming) 功能的使用方法和效果。

## 1. 环境配置

In [ ]:
# 导入必要的库
import requests
import json
import time
from typing import Iterator, Optional, Tuple
from IPython.display import display, clear_output, Markdown

# API 配置
API_KEY = "f7ec61a7856a4f4389e037eb588c69df.0pZgcFlTCk9GHCXM"
BASE_URL = "https://open.bigmodel.cn/api/paas/v4/chat/completions"

print("✅ 环境配置完成")
print(f"API 端点: {BASE_URL}")

### 1.1 流式响应解析封装

In [ ]:
def stream_chat_completion(
    model: str, 
    messages: list, 
    api_key: str = API_KEY,
    show_realtime: bool = True
) -> Tuple[str, Optional[str], float]:
    """
    流式对话完成
    
    Args:
        model: 模型代码
        messages: 消息列表
        api_key: API 密钥
        show_realtime: 是否实时显示输出
    
    Returns:
        (完整内容, 思维链内容, 总耗时秒)
    """
    start_time = time.time()
    
    response = requests.post(
        url=BASE_URL,
        headers={
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        },
        json={
            "model": model,
            "messages": messages,
            "stream": True
        },
        stream=True,
        timeout=(10, 300)
    )
    
    response.raise_for_status()
    
    full_content = ""
    full_reasoning = ""
    first_token_time = None
    
    if show_realtime:
        print(f"\n🤖 [{model}] 实时输出:\n" + "-"*50)
        print("💬 ", end="", flush=True)
    
    for line in response.iter_lines():
        if not line:
            continue
        
        line = line.decode('utf-8')
        
        if not line.startswith('data: '):
            continue
        
        data = line[6:]  # 去掉 "data: " 前缀
        
        # 检查结束标记
        if data == '[DONE]':
            if show_realtime:
                print(f"\n" + "-"*50)
            break
        
        try:
            chunk = json.loads(data)
            choice = chunk.get('choices', [{}])[0]
            delta = choice.get('delta', {})
            
            # 记录首 token 时间
            if first_token_time is None:
                first_token_time = time.time() - start_time
            
            # 提取内容
            content = delta.get('content', '')
            if content:
                full_content += content
                if show_realtime:
                    print(content, end="", flush=True)
            
            # 提取思维链（仅 Thinking 模型）
            reasoning = delta.get('reasoning_content', '')
            if reasoning:
                full_reasoning += reasoning
                
        except json.JSONDecodeError:
            continue
    
    total_time = time.time() - start_time
    
    if show_realtime:
        print(f"\n\n⏱️ 首 token 延迟: {first_token_time:.3f}s")
        print(f"⏱️ 总耗时: {total_time:.3f}s")
        print(f"📝 总字符数: {len(full_content)}")
    
    return full_content, full_reasoning if full_reasoning else None, total_time


def non_stream_chat_completion(
    model: str, 
    messages: list, 
    api_key: str = API_KEY
) -> Tuple[str, Optional[str], float]:
    """非流式对话完成（用于对比）"""
    start_time = time.time()
    
    response = requests.post(
        url=BASE_URL,
        headers={
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        },
        json={
            "model": model,
            "messages": messages,
            "stream": False
        },
        timeout=(10, 300)
    )
    
    response.raise_for_status()
    result = response.json()
    total_time = time.time() - start_time
    
    choice = result.get('choices', [{}])[0]
    message = choice.get('message', {})
    content = message.get('content', '')
    reasoning = message.get('reasoning_content', '')
    
    return content, reasoning if reasoning else None, total_time

print("✅ 封装函数定义完成")

---

## 2. GLM-4.7-Flash 流式输出测试

**模型信息：**
- 模型代码：`glm-4.7-flash`
- 上下文：200K
- 最大输出：128K
- 特点：超长上下文，适合长文档处理

In [ ]:
# 测试：简单问候
messages = [{"role": "user", "content": "你好，请简短介绍一下自己"}]
content, reasoning, duration = stream_chat_completion("glm-4.7-flash", messages)

print(f"\n📦 思维链: {reasoning if reasoning else '无'}")

In [ ]:
# 测试：长文本生成
messages = [{"role": "user", "content": "写一篇关于人工智能的短文（约200字）"}]
content, reasoning, duration = stream_chat_completion("glm-4.7-flash", messages)

---

## 3. GLM-4-Flash-250414 流式输出测试

**模型信息：**
- 模型代码：`glm-4-flash-250414`
- 上下文：128K
- 最大输出：16K
- 特点：轻量级，响应速度快

In [ ]:
# 测试：代码生成
messages = [{
    "role": "user", 
    "content": "用 Python 写一个计算斐波那契数列的函数，并加上注释"
}]
content, reasoning, duration = stream_chat_completion("glm-4-flash-250414", messages)

In [ ]:
# 测试：多轮对话
messages = [
    {"role": "user", "content": "什么是机器学习？"},
    {"role": "assistant", "content": "机器学习是人工智能的一个分支..."},
    {"role": "user", "content": "能举个例子吗？"}
]
content, reasoning, duration = stream_chat_completion("glm-4-flash-250414", messages)

---

## 4. GLM-4.6V-Flash 流式输出测试（视觉模型）

**模型信息：**
- 模型代码：`glm-4.6v-flash`
- 上下文：128K
- 最大输出：32K
- 特点：支持图片输入，自动判断是否思考

In [ ]:
def stream_vision_chat(
    model: str,
    image_url: str,
    prompt: str,
    api_key: str = API_KEY
) -> Tuple[str, float]:
    """视觉模型流式对话"""
    messages = [{
        "role": "user",
        "content": [
            {"type": "image_url", "image_url": {"url": image_url}},
            {"type": "text", "text": prompt}
        ]
    }]
    
    content, _, duration = stream_chat_completion(model, messages, api_key)
    return content, duration

print("✅ 视觉模型封装完成")

In [ ]:
# 测试：使用示例图片
# 使用智谱官方示例图片
sample_image = "https://img.alicdn.com/imgextra/i1/O1CN01xRdqwB1M3UOxVL2sO_!!6000000001380-2-tps-1200-1200.png"

content, duration = stream_vision_chat(
    "glm-4.6v-flash",
    sample_image,
    "请描述这张图片的内容"
)

print(f"\n📷 视觉理解完成，耗时: {duration:.3f}s")

In [ ]:
# 测试：基于图片的推理
content, duration = stream_vision_chat(
    "glm-4.6v-flash",
    sample_image,
    "这张图片可能用于什么场景？"
)

---

## 5. GLM-4.1V-Thinking-Flash 流式输出测试（思考模型）

**模型信息：**
- 模型代码：`glm-4.1v-thinking-flash`
- 上下文：64K
- 最大输出：16K
- 特点：**支持思维链 (reasoning_content)**，可展示推理过程

In [ ]:
# 测试：数学问题（带思维链展示）
messages = [{"role": "user", "content": "解方程：3x + 7 = 22，请展示思考过程"}]

print("🔢 数学问题求解\n" + "="*60)
content, reasoning, duration = stream_chat_completion("glm-4.1v-thinking-flash", messages)

print("\n" + "="*60)
print(f"\n🧠 思维链（推理过程）:\n{reasoning}")
print(f"\n💡 最终答案:\n{content}")

In [ ]:
# 测试：逻辑推理
messages = [{
    "role": "user", 
    "content": """
    逻辑推理题：
    有三个人 A、B、C，分别来自北京、上海、广州。
    已知：
    1. A 不是北京人
    2. B 不是上海人
    3. 北京人不是最年轻的
    4. B 年龄最大
    请问每个人来自哪个城市？
    """
}]

content, reasoning, duration = stream_chat_completion("glm-4.1v-thinking-flash", messages)

print("\n" + "="*60)
print(f"\n🧠 思维链:\n{reasoning}")
print(f"\n💡 答案:\n{content}")

---

## 6. 流式响应解析封装（高级）

In [ ]:
class StreamingParser:
    """流式响应解析器"""
    
    @staticmethod
    def parse_sse_line(line: bytes) -> Optional[dict]:
        """解析单行 SSE 数据"""
        if not line:
            return None
        
        decoded = line.decode('utf-8')
        
        # 跳过空行和注释
        if not decoded or decoded.startswith(':') or not decoded.startswith('data: '):
            return None
        
        data = decoded[6:]  # 去掉 "data: "
        
        # 检查结束标记
        if data == '[DONE]':
            return {'done': True}
        
        try:
            return json.loads(data)
        except json.JSONDecodeError:
            return None
    
    @staticmethod
    def extract_delta(chunk: dict) -> dict:
        """从 chunk 中提取增量信息"""
        result = {
            'content': '',
            'reasoning_content': '',
            'role': None,
            'finish_reason': None
        }
        
        if not chunk or 'choices' not in chunk:
            return result
        
        choice = chunk['choices'][0]
        delta = choice.get('delta', {})
        
        result['content'] = delta.get('content', '')
        result['reasoning_content'] = delta.get('reasoning_content', '')
        result['role'] = delta.get('role')
        result['finish_reason'] = choice.get('finish_reason')
        
        return result


# 测试解析器
parser = StreamingParser()

# 测试用例
test_lines = [
    b'data: {"choices": [{"delta": {"role": "assistant"}}]}',
    b'data: {"choices": [{"delta": {"content": "你好"}}]}',
    b'data: {"choices": [{"delta": {"content": "世界"}}]}',
    b'data: [DONE]'
]

print("📋 解析器测试:\n" + "-"*40)
for line in test_lines:
    parsed = parser.parse_sse_line(line)
    if parsed:
        if parsed.get('done'):
            print("✅ 检测到结束标记: [DONE]")
        else:
            delta = parser.extract_delta(parsed)
            print(f"内容: '{delta['content']}' | 思维链: '{delta['reasoning_content']}'")

print("\n✅ 解析器工作正常")

---

## 7. 性能对比：流式 vs 非流式

In [ ]:
# 性能对比测试
import pandas as pd

test_prompt = "请详细解释量子计算的基本原理和应用前景（约300字）"
messages = [{"role": "user", "content": test_prompt}]

models = [
    ("glm-4.7-flash", "GLM-4.7-Flash"),
    ("glm-4-flash-250414", "GLM-4-Flash-250414"),
]

results = []

print("⚡ 性能对比测试开始\n" + "="*60)

for model_code, model_name in models:
    print(f"\n📊 测试模型: {model_name}")
    
    # 流式测试
    print("  🌊 流式模式...", end="")
    _, _, stream_time = stream_chat_completion(
        model_code, messages, show_realtime=False
    )
    print(f" 耗时: {stream_time:.3f}s")
    
    # 非流式测试
    print("  📦 非流式模式...", end="")
    _, _, non_stream_time = non_stream_chat_completion(model_code, messages)
    print(f" 耗时: {non_stream_time:.3f}s")
    
    results.append({
        '模型': model_name,
        '流式耗时(s)': round(stream_time, 3),
        '非流式耗时(s)': round(non_stream_time, 3),
        '差异(s)': round(non_stream_time - stream_time, 3)
    })

# 显示结果表格
df = pd.DataFrame(results)
print("\n" + "="*60)
print("📈 性能对比结果:")
print(df.to_string(index=False))

print("\n💡 说明:")
print("   - 流式耗时：包含首 token 到结束的完整时间")
print("   - 非流式耗时：等待完整响应的时间")
print("   - 实际体验：流式模式首 token 延迟更低，用户体验更好")

---

## 8. 错误处理测试

In [ ]:
# 错误处理测试
def test_error_handling():
    """测试各种错误情况"""
    
    print("🧪 错误处理测试\n" + "="*60)
    
    # 测试 1：无效的 API Key
    print("\n❌ 测试 1: 无效的 API Key")
    try:
        _, _, _ = stream_chat_completion(
            "glm-4.7-flash",
            [{"role": "user", "content": "你好"}],
            api_key="invalid_key",
            show_realtime=False
        )
    except requests.exceptions.HTTPError as e:
        print(f"   ✅ 正确捕获错误: {e.response.status_code}")
    except Exception as e:
        print(f"   ⚠️ 其他错误: {type(e).__name__}")
    
    # 测试 2：无效的模型
    print("\n❌ 测试 2: 无效的模型代码")
    try:
        _, _, _ = stream_chat_completion(
            "invalid-model-name",
            [{"role": "user", "content": "你好"}],
            show_realtime=False
        )
    except requests.exceptions.HTTPError as e:
        print(f"   ✅ 正确捕获错误: {e.response.status_code}")
        if e.response.status_code == 400:
            print("   ℹ️  400 错误：模型不存在或参数错误")
    except Exception as e:
        print(f"   ⚠️ 其他错误: {type(e).__name__}")
    
    # 测试 3：空消息
    print("\n❌ 测试 3: 空消息")
    try:
        _, _, _ = stream_chat_completion(
            "glm-4.7-flash",
            [],
            show_realtime=False
        )
    except requests.exceptions.HTTPError as e:
        print(f"   ✅ 正确捕获错误: {e.response.status_code}")
    except Exception as e:
        print(f"   ⚠️ 其他错误: {type(e).__name__}: {e}")
    
    # 测试 4：超长消息
    print("\n❌ 测试 4: 超长消息（超出上下文限制）")
    try:
        long_content = "这是一个测试。" * 50000  # 约 15 万字
        _, _, _ = stream_chat_completion(
            "glm-4v-flash",  # 只有 16K 上下文
            [{"role": "user", "content": long_content}],
            show_realtime=False
        )
    except requests.exceptions.HTTPError as e:
        print(f"   ✅ 正确捕获错误: {e.response.status_code}")
    except Exception as e:
        print(f"   ⚠️ 其他错误: {type(e).__name__}")
    
    print("\n" + "="*60)
    print("✅ 错误处理测试完成")

test_error_handling()

---

## 9. 完整流式客户端封装

In [ ]:
class ZhipuStreamClient:
    """
    智谱 AI 流式输出客户端
    
    支持所有免费模型：
    - glm-4.7-flash: 200K 上下文，超长文本
    - glm-4-flash-250414: 128K 上下文，轻量快速
    - glm-4.6v-flash: 视觉理解，自动思考
    - glm-4.1v-thinking-flash: 视觉 + 思维链
    - glm-4v-flash: 基础视觉模型
    """
    
    BASE_URL = "https://open.bigmodel.cn/api/paas/v4/chat/completions"
    
    # 模型配置
    MODELS = {
        "glm-4.7-flash": {
            "context": "200K",
            "max_output": "128K",
            "thinking": False,
            "vision": False,
            "description": "超长上下文文本模型"
        },
        "glm-4-flash-250414": {
            "context": "128K",
            "max_output": "16K",
            "thinking": False,
            "vision": False,
            "description": "轻量级快速文本模型"
        },
        "glm-4.6v-flash": {
            "context": "128K",
            "max_output": "32K",
            "thinking": "auto",
            "vision": True,
            "description": "视觉理解模型（自动思考）"
        },
        "glm-4.1v-thinking-flash": {
            "context": "64K",
            "max_output": "16K",
            "thinking": True,
            "vision": True,
            "description": "视觉思考模型（支持思维链）"
        },
        "glm-4v-flash": {
            "context": "16K",
            "max_output": "1K",
            "thinking": False,
            "vision": True,
            "description": "基础视觉模型"
        }
    }
    
    def __init__(self, api_key: str = API_KEY):
        self.api_key = api_key
        self.session = requests.Session()
    
    def chat(
        self,
        model: str,
        content: str,
        image_url: str = None,
        show_stream: bool = True,
        show_reasoning: bool = True
    ) -> dict:
        """
        流式对话
        
        Args:
            model: 模型代码
            content: 文本内容
            image_url: 图片 URL（视觉模型使用）
            show_stream: 是否实时显示输出
            show_reasoning: 是否显示思维链
        
        Returns:
            dict: {content, reasoning, duration, first_token_time}
        """
        # 构建消息
        if image_url:
            messages = [{
                "role": "user",
                "content": [
                    {"type": "image_url", "image_url": {"url": image_url}},
                    {"type": "text", "text": content}
                ]
            }]
        else:
            messages = [{"role": "user", "content": content}]
        
        # 发送请求
        start_time = time.time()
        first_token_time = None
        
        response = self.session.post(
            url=self.BASE_URL,
            headers={
                "Authorization": f"Bearer {self.api_key}",
                "Content-Type": "application/json"
            },
            json={
                "model": model,
                "messages": messages,
                "stream": True
            },
            stream=True,
            timeout=(10, 300)
        )
        
        response.raise_for_status()
        
        # 解析流式响应
        full_content = []
        full_reasoning = []
        
        if show_stream:
            model_info = self.MODELS.get(model, {})
            thinking_support = model_info.get('thinking')
            print(f"\n🤖 [{model}]")
            print(f"   {model_info.get('description', '')}")
            if thinking_support:
                print(f"   💭 思维链: {'支持' if thinking_support == True else '自动'}")
            print("-" * 50)
        
        for line in response.iter_lines():
            if not line:
                continue
            
            parsed = StreamingParser.parse_sse_line(line)
            if not parsed:
                continue
            
            if parsed.get('done'):
                break
            
            delta = StreamingParser.extract_delta(parsed)
            
            if first_token_time is None and delta['content']:
                first_token_time = time.time() - start_time
            
            if delta['content']:
                full_content.append(delta['content'])
                if show_stream:
                    print(delta['content'], end='', flush=True)
            
            if delta['reasoning_content']:
                full_reasoning.append(delta['reasoning_content'])
        
        duration = time.time() - start_time
        result_content = ''.join(full_content)
        result_reasoning = ''.join(full_reasoning) if full_reasoning else None
        
        if show_stream:
            print(f"\n\n" + "-" * 50)
            print(f"⏱️ 首 token: {first_token_time:.3f}s | 总耗时: {duration:.3f}s")
            if result_reasoning and show_reasoning:
                print(f"\n🧠 思维链:\n{result_reasoning}")
        
        return {
            'content': result_content,
            'reasoning': result_reasoning,
            'duration': duration,
            'first_token_time': first_token_time
        }
    
    def list_models(self):
        """显示所有支持的模型"""
        print("📋 支持的模型列表:\n" + "=" * 60)
        for code, info in self.MODELS.items():
            print(f"\n🔹 {code}")
            print(f"   描述: {info['description']}")
            print(f"   上下文: {info['context']} | 最大输出: {info['max_output']}")
            print(f"   视觉: {'✅' if info['vision'] else '❌'} | 思考: {'✅' if info['thinking'] else '❌' if info['thinking'] == False else '🤖'}")


# 初始化客户端
client = ZhipuStreamClient()
client.list_models()

In [ ]:
# 使用客户端测试
print("\n" + "="*60)
print("🧪 客户端功能测试\n")

# 文本模型
result = client.chat(
    model="glm-4.7-flash",
    content="请用一句话总结人工智能的发展趋势"
)

In [ ]:
# 思考模型
result = client.chat(
    model="glm-4.1v-thinking-flash",
    content="计算 99 * 99，并展示思考过程"
)

---

## 总结

本 Notebook 完整演示了智谱 AI 流式输出功能：

### ✅ 已完成测试
1. **GLM-4.7-Flash** - 超长上下文文本生成
2. **GLM-4-Flash-250414** - 轻量级快速响应
3. **GLM-4.6V-Flash** - 视觉理解能力
4. **GLM-4.1V-Thinking-Flash** - 思维链展示
5. **SSE 解析封装** - 完整的流式响应处理
6. **性能对比** - 流式 vs 非流式
7. **错误处理** - 各种异常情况处理

### 🔑 关键要点
- 使用 `stream=true` 启用流式输出
- SSE 数据以 `data: ` 开头，以 `data: [DONE]` 结束
- 解析增量内容使用 `choices[0].delta.content`
- Thinking 模型额外返回 `reasoning_content`
- 流式输出首 token 延迟低，用户体验更好